# Linear Regression applied to "Mitad" exercise


In [1]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import spacy
#from deda_utils import extract_metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Load Mitad Submissions Dataset

We decided to work with submissions that are part of the collection *failed_submissions.* We took two filter decisitions. First, we filter the submissions per language. Then we filter the submissions by exercise. 

As first approach, we decide to work with a particular Haskell exercise. We use the [**Mitad** exercise](https://central.mumuki.io/exercises/4256-programacion-funcional-practica-valores-y-funciones-mitad). 


In [9]:
dataset_home = "datasets/haskell"
json_file = open(os.path.join(dataset_home, 'submissions_Recortar tuits.json'), encoding='utf8')
json_data = json.load(json_file)
submissions = pd.io.json.json_normalize(json_data)


NameError: name 'file' is not defined

# Cleaning the Dataset

We droped all the submissions with **'pending'** or **'aborted' **status, because them don't provide significative data about the code and the feedback is related with server or technical problems of the Mumuki platform.
Also we droped submissions with empty content because them would cause issues with vectorizer. At the moment we take this decistion, we suppose that submissions that have empty content were related with Mumuki exercises that don't need code to pass them. For example, this [Gobstones exercise](https://central.mumuki.io/exercises/259-fundamentos-primeros-programas-el-tablero). But the mitad exercise is not the case. So, why we definetly decide to eliminate them? First, empty content generate problems when we try yo use the vectorizer. Second, it's a border case that  we prefer not to consider in this first approach.

Why we choose **mitad** exercise for this first approach? As first, we think that should be interesting to choose an exercise that have a great percentage of pass submissions. 

We calculate the distributions of submissions status. *extract_metrics* is a function defined by us that calculate the distribution of submissions status.


In [8]:
submissions[submissions['exercise.name'] == 'Recortar tuits'].columns

Index(['course', 'created_at', 'exercise.eid', 'exercise.name',
       'exercise.number', 'guide.language.name', 'guide.lesson.id',
       'guide.lesson.name', 'guide.name', 'guide.parent.chapter.id',
       'guide.parent.chapter.name', 'guide.parent.name',
       'guide.parent.position', 'guide.parent.type', 'guide.slug',
       'organization', 'student.email', 'student.first_name',
       'student.image_url', 'student.last_name', 'student.name',
       'student.social_id', 'student.uid', 'submissions', 'updated_at'],
      dtype='object')

In [11]:
submissions[submissions['status'] == 'passed']['content'][9]

'limitarCaracteres (x,y) =  (x, take 15 y)\r\nrecortar listaTuits = map limitarCaracteres listaTuits\r\nrecortar::[(String,String)] -> [(String,String)]'

In [4]:
submissions = submissions[submissions['content'] != ""]
submissions = submissions[~submissions['content'].isnull()]
submissions = submissions[~(submissions['status'] == 'aborted')]
submissions = submissions[~(submissions['status'] == 'pending')]

metrics = extract_metrics(submissions,'status')
print(metrics)

NameError: name 'extract_metrics' is not defined

In [23]:
submissions.columns

Index(['_id.$oid', 'content', 'created_at.$date', 'exercise.bibliotheca_id',
       'exercise.eid', 'exercise.name', 'exercise.number',
       'expectation_results', 'feedback', 'guide.chapter.id',
       'guide.chapter.name', 'guide.language.name', 'guide.lesson.number',
       'guide.name', 'guide.parent.chapter.id', 'guide.parent.chapter.name',
       'guide.parent.name', 'guide.parent.position', 'guide.parent.type',
       'guide.slug', 'id', 'manual_evaluation_comment', 'organization',
       'queries', 'query_results', 'result', 'sid', 'status',
       'submissions_count', 'submitter.email',
       'submitter.email;first_name;last_name', 'submitter.first_name',
       'submitter.id', 'submitter.image_url', 'submitter.last_name',
       'submitter.name', 'submitter.social_id', 'submitter.uid',
       'test_results', 'result_custom'],
      dtype='object')

# Differents Test Results

In [9]:
submissions[submissions['status']=='failed']['test_results'][4]

[{'title': 'recortar recortar [] == []',
  'status': 'failed',
  'result': '/tmp/compilation58:8:27:\n    Could not deduce (b ~ [a0])\n    from the context (Show a, Show b)\n      bound by the type signature for\n                 recortar :: (Show a, Show b) => [(a, b)] -> [(a, b)]\n      at /tmp/compilation58:9:11-48\n      ‘b’ is a rigid type variable bound by\n          the type signature for\n            recortar :: (Show a, Show b) => [(a, b)] -> [(a, b)]\n          at /tmp/compilation58:9:11\n    Expected type: (a, [a0]) -> (a, b)\n      Actual type: (a, [a0]) -> (a, [a0])\n    Relevant bindings include\n      listaTuits :: [(a, b)] (bound at /tmp/compilation58:8:10)\n      recortar :: [(a, b)] -> [(a, b)] (bound at /tmp/compilation58:8:1)\n    In the first argument of ‘map’, namely ‘limitarCaracteres’\n    In the expression: map limitarCaracteres listaTuits\n(deferred type error)'},
 {'title': 'recortar recortar [("@lorem", "Lorem ipsum dolor sit amet, consectetur adipiscing eli

In [85]:
submissions[submissions['status']=='failed']['test_results'][39]

[{'title': 'mitad mitad 4 == 2',
  'status': 'failed',
  'result': '/tmp/compilation58:13:13:\n    No instance for (Show (a0 -> a0)) arising from a use of ‘shouldBe’\n    In a stmt of a \'do\' block: mitad 4 `shouldBe` 2\n    In the second argument of ‘($)’, namely\n      ‘do { mitad 4 `shouldBe` 2 }’\n    In a stmt of a \'do\' block:\n      it "mitad 4 == 2" $ do { mitad 4 `shouldBe` 2 }\n(deferred type error)'},
 {'title': 'mitad mitad 9 == 4.5',
  'status': 'failed',
  'result': '/tmp/compilation58:16:13:\n    No instance for (Show (a1 -> a1)) arising from a use of ‘shouldBe’\n    In a stmt of a \'do\' block: mitad 9 `shouldBe` 4.5\n    In the second argument of ‘($)’, namely\n      ‘do { mitad 9 `shouldBe` 4.5 }’\n    In a stmt of a \'do\' block:\n      it "mitad 9 == 4.5" $ do { mitad 9 `shouldBe` 4.5 }\n(deferred type error)'},
 {'title': 'mitad mitad 10 == 5',
  'status': 'failed',
  'result': '/tmp/compilation58:19:14:\n    No instance for (Show (a2 -> a2)) arising from a use o

In [68]:
submissions[submissions['status']=='failed']['test_results'][1558]

[{'title': 'mitad mitad 4 == 2',
  'status': 'failed',
  'result': 'expected: 2.0\n but got: 0.5'},
 {'title': 'mitad mitad 9 == 4.5',
  'status': 'failed',
  'result': 'expected: 4.5\n but got: 0.2222222222222222'},
 {'title': 'mitad mitad 10 == 5',
  'status': 'failed',
  'result': 'expected: 5.0\n but got: 0.2'}]

# Differents Passed with warnigns expectation results

In [69]:
submissions[submissions['status']=='passed_with_warnings']['expectation_results']

1179    [{'binding': 'mitad', 'inspection': 'HasRedund...
1395    [{'binding': 'mitad', 'inspection': 'HasRedund...
Name: expectation_results, dtype: object

In [60]:
submissions[submissions['status']=='passed_with_warnings']['expectation_results'][1179]

[{'binding': 'mitad',
  'inspection': 'HasRedundantParameter',
  'result': 'failed'}]

In [70]:
submissions[submissions['status']=='passed_with_warnings']['expectation_results'][1395]

[{'binding': 'mitad',
  'inspection': 'HasRedundantParameter',
  'result': 'failed'}]

### Anotaciones
En result estamos guardando los test_results en caso de que el status sea failed. y los expectations result en caso que sea passed_with_warnigns falta de expectativas.
El problema es que la salida de los test (a.k.a test_results) como es la salida del "compilador" varia segun la forma en la que falla.
Ergo hay 118 clases distintas para clasificar
ver que label poner en caso que falle los test

In [10]:
submissions['result'] =  submissions.apply(
    lambda x : str(x.test_results) if x.status == 'failed' else (
        str(x.expectation_results) if x.status == 'passed_with_warnings' else('passed' if x.status == 'passed' else x.result)), axis=1)

submissions['result'] =  submissions.apply(
    lambda x : str('Error de sintaxis') if x.status == 'errored' else x.result, axis=1)
    
    
submissions['result_custom'] = submissions['status'] + ' ' + submissions['result']


# Show Distribution of Submissions Status

Our dataset has the following distribution:

|  Status | %  |
|-|-|
| Errored  |  0.097099|
|  Failed |  0.194198 |
|  Passed |  0.707519 |
|  Passed WW | 0.001184  |

# Train, Dev and Test Set

In this particular case we decided to split our dataset only in train and test. We don't use dev set because the number of submissions is small.

The train set is composed by **1131 submissions**, while the Test set have ** 558 submissions**.

In [11]:
X = submissions['content']
Y = submissions['result_custom']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

# Distributions of status Submissions in Train and Test Set

We try to maintain the same status distribution in train and test set as a good practice to train a machine learning model. Test and Train sets have at least one example of each status to make sure when we will train/test our model with train set it contains examples of all types of status.

In [12]:
Y_train.value_counts()/Y_train.shape[0]

errored Error de sintaxis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Distributions of status Submissions in Test set

In [13]:
Y_test.value_counts()/Y_test.shape[0]

errored Error de sintaxis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Choose Model and Training
We choosed Linear Regression as first approach model because is the easiest model to train. We select it as our baseline, and will try to obtain the best performance classifing submissions setting parameters and selecting tokenizer. In case that not obtain a good performance we are going to change ML model. 
As we describe in ** Cleaning the Dataset** section, we use *CountVectorizer* for convert submissions content into features and *LabelEncoder* to convert an status into a number. ** Status**. On this first approach the **status** will be target for our machine learning model.


In [30]:
cv = CountVectorizer(lowercase=False)
X_transformed = cv.fit_transform(['limitarCaracteres (x,y) =  (x, take 15 y)\r\nrecortar listaTuits = map limitarCaracteres listaTuits\r\nrecortar::[(a,b)] -> [a]'])

In [15]:
le = LabelEncoder()
Y_transformed = le.fit_transform(Y_train)

In [16]:
len(le.classes_)

2224

# OneVsRest Classifier
As we have to train a multiclass classifier, we need to use [one-vs-rest algorithm](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest). In a few words, create a binary classifier (Linear Regresssion) for each status type, classify the example by all the classifiers and get the max of those predictions as predicted label.

In [20]:
lr = OneVsRestClassifier(LinearRegression())c
lr.fit(X_transformed, Y_transformed)

OneVsRestClassifier(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
          n_jobs=1)

In [21]:
predicted = lr.predict(cv.transform(X_test))

# Calculate Metrics
Below we show precision recall and f1-score of classifier.

In [22]:
print(classification_report(le.transform(Y_test), predicted, target_names=le.classes_, digits=2))

ValueError: y contains new labels: ['failed [{\'title\': \'mitad mitad 4 == 2\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(deferred type error)"}, {\'title\': \'mitad mitad 9 == 4.5\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(deferred type error)"}, {\'title\': \'mitad mitad 10 == 5\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(d

# Count Vectorizer with Spacy Tokenizer

Use CountVectorizer with [spacy](https://spacy.io/) tokenizer as alternative.

In [31]:
cv_custom = CountVectorizer(tokenizer=spacy.load('en_core_web_sm'), lowercase=False)
X_transformed = cv_custom.fit_transform(['limitarCaracteres (x,y) =  (x, take 15 y)\r\nrecortar listaTuits = map limitarCaracteres listaTuits\r\nrecortar::[(a,b)] -> [a]'])

In [24]:
le = LabelEncoder()
Y_transformed = le.fit_transform(Y_train)

In [25]:
lr_custom = OneVsRestClassifier(LinearRegression())
lr_custom.fit(X_transformed, Y_transformed)

OneVsRestClassifier(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
          n_jobs=1)

In [26]:
predicted_custom = lr_custom.predict(cv_custom.transform(X_test))

In [27]:
print(classification_report(le.transform(Y_test), predicted_custom, target_names=le.classes_, digits=2))

ValueError: y contains new labels: ['failed [{\'title\': \'mitad mitad 4 == 2\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(deferred type error)"}, {\'title\': \'mitad mitad 9 == 4.5\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(deferred type error)"}, {\'title\': \'mitad mitad 10 == 5\', \'status\': \'failed\', \'result\': "/tmp/mumuki.compile20160715-9641-ynmxz1.hs:25:13:\\n    Couldn\'t match expected type 8216Float8217 with actual type 8216Int8217\\n    In the first argument of 8216(/)8217, namely 8216num8217\\n    In the expression: num / 2\\n(d

# Analyze difference between vectorizer
Below we show the vocabulary obtained in base of vectorizer.
Using default tokenizer, numbers and math operator was dropped this will represent a problem in future because the semantics of a program depends on it. On the oder hand, using spacy tokenizer math operators and numbers are treated like a token but this tokenizer duplice tokens like : , 2. We show this situation below.

## Vocabulary countvectorizer with default tokenizer

In [ ]:
cv.vocabulary_

## Vocabulary countvectorizer with spacy tokenizer

In [ ]:
cv_custom.vocabulary_

# Decisions about tokenizer
Based on the average of all metrics that we obtained, we decide to use countVectorizer with default tokenizer,  because it have better performance. In the future we will use a language based tokenizer.

# Improve Performance
In order to improve performance in classifier we decide to test another model we will use SVM.
We are going to test 3 diferents kernels, Linear, Polynomial and RBF

In [14]:
metrics_kernel = {}
for kernel in ('linear', 'poly', 'rbf'):
    clf = SVC(kernel=kernel, gamma=2)
    clf.fit(X_transformed, Y_transformed)
    score = clf.score(cv.transform(X_test), le.transform(Y_test))
    metrics_kernel[kernel] = score

In [15]:
metrics_kernel

{'linear': 0.8440860215053764,
 'poly': 0.8709677419354839,
 'rbf': 0.8673835125448028}

we are going to use gridsearch to obtain the best parameter configuration for SVC. The cell below take a lot of time to be executed.

In [22]:
parameters = {
    'kernel':('linear', 'rbf', 'poly'),
    'C':[0.01, 0.1, 1, 10, 100, 1000],
    'gamma':[0.1, 0.5, 1, 2],
    'degree':[1,2,3,4, 5]}
svc = SVC()
clf = GridSearchCV(svc, parameters, n_jobs=4)
clf.fit(X_transformed, Y_transformed)

/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'kernel': ('linear', 'rbf', 'poly'), 'C': [0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.1, 0.5, 1, 2], 'degree': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
clf.best_params_

{'C': 100, 'degree': 1, 'gamma': 0.1, 'kernel': 'rbf'}

Obtain the mean accuracy on the given test data and labels with the best parameters obtained in gridsearch. We obtained a similar performance that we obtained using linearregression, only 1.6% better accuracy

In [24]:
clf.score(cv.transform(X_test), le.transform(Y_test))

0.8763440860215054

In [17]:
transformed = cv.transform(['limitarCaracteres (x,y) =  (x, take 15 y)\r\nrecortar listaTuits = map limitarCaracteres listaTuits\r\nrecortar::[(a,b)] -> [a]'])

In [21]:
cv.vocabulary_

{'recortar': 328,
 'map': 226,
 'aux': 57,
 'snd': 413,
 'take': 428,
 '15': 3,
 'Ej': 9,
 'lista': 199,
 'de': 119,
 'tuits': 486,
 'elBarto': 134,
 'yo': 526,
 'no': 259,
 'fui': 159,
 'krusty': 180,
 'tampoco': 435,
 'funcionAuxiliar': 164,
 'unTuit': 499,
 'fst': 158,
 'listaTuits': 204,
 'La': 14,
 'función': 168,
 'recibe': 308,
 'una': 503,
 'tuplas': 488,
 'elementos': 141,
 'léase': 224,
 'la': 181,
 'mapea': 229,
 'con': 84,
 'funciónAuxiliar': 169,
 'Al': 6,
 'mapearla': 230,
 'debería': 120,
 'obtener': 265,
 'como': 82,
 'devuelve': 127,
 'tupla': 487,
 'del': 123,
 'mismo': 251,
 'tipo': 443,
 'pero': 279,
 'mas': 232,
 'corta': 93,
 'recortarUnTuit': 354,
 'tuit': 477,
 'length': 185,
 'String': 25,
 'usuario': 516,
 'recortarUno': 356,
 'cut': 114,
 'otherwise': 272,
 'listaDeTuits': 201,
 'listDeTuits': 196,
 'truncar': 455,
 'recortarTuits': 347,
 'cad1': 73,
 'cad2': 74,
 'recortarTuit': 346,
 'limitar': 188,
 'recortarUnoSolo': 357,
 't1': 425,
 't2': 426,
 'minimum

In [23]:
cv2 = CountVectorizer()
cv2.fit_transform(['limitarCaracteres (x,y) =  (x, take 15 y)\r\nrecortar listaTuits = map limitarCaracteres listaTuits\r\nrecortar::[(a,b)] -> [a]'])

<1x6 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [25]:
cv.vocabulary_

{'limitarCaracteres': 1,
 'take': 5,
 '15': 0,
 'recortar': 4,
 'listaTuits': 2,
 'map': 3}

In [33]:
cv_custom.vocabulary_

{limitarCaracteres: 0,
 (: 1,
 x: 2,
 ,: 3,
 y: 4,
 ): 5,
 =: 6,
  : 7,
 (: 8,
 x: 9,
 ,: 10,
 take: 11,
 15: 12,
 y: 13,
 ): 14,
 : 15,
 recortar: 16,
 listaTuits: 17,
 =: 18,
 map: 19,
 limitarCaracteres: 20,
 listaTuits: 21,
 : 22,
 recortar::[(a: 23,
 ,: 24,
 b: 25,
 ): 26,
 ]: 27,
 -: 28,
 >: 29,
 [: 30,
 a: 31,
 ]: 32}

In [32]:
cv.vocabulary_

{'limitarCaracteres': 1,
 'take': 5,
 '15': 0,
 'recortar': 4,
 'listaTuits': 2,
 'map': 3}